# Combining original and flipped segmentations to get purely right-ventricle segmentations

In [1]:
from pathlib import Path
import sys

import cv2
import numpy as np
from tqdm import tqdm

import echonet

# print(cv2.getBuildInformation())

In [7]:
LEFT_SEGMENT_DIR = Path("output/segmentation/all-patients")
RIGHT_SEGMENT_DIR = Path("output/segmentation/flipped")
ECHONET_VIDEO_DIR = Path("/home/lex/data/echonet-data/Videos")
OUTPUT_DIR = Path("output/right-only-segmentation")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RED = np.array([255, 0, 0])
GREEN = np.array([0, 255, 0])
BLUE = np.array([0, 0, 255])
ORANGE = np.array([255, 165, 0])
LIGHT_GREY = np.array([211, 211, 211])

In [3]:
def get_heights(mask: np.ndarray):
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index

        if len(row_indices[this_frame]) == 0:
            # print(f"Frame #{frame_index} appears to have no segmentations? Treating this as zero height...")
            heights.append(0)
            continue

        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

In [4]:
left_segment_mask_fps = [p for p in (LEFT_SEGMENT_DIR / "segmentation_masks").iterdir()]
for left_segment_mask_fp in tqdm(left_segment_mask_fps):
    # Get associated segment masks for this particular video
    # video_mask_name = video_fp.with_suffix(".npy").name # Turns "path/to/video.avi" into "video.npy"
    video_stem = Path(left_segment_mask_fp.stem) # turns "path/to/0x12345.npy" into just "0x12345"
    right_segment_mask_fp = RIGHT_SEGMENT_DIR / "segmentation_masks" / video_stem.with_suffix(".npy")
    echonet_video_fp = ECHONET_VIDEO_DIR / video_stem.with_suffix(".avi")
    # print(f"Processing {echonet_video_fp}...")

    left_segment_mask = np.load(left_segment_mask_fp)
    right_segment_mask = np.load(right_segment_mask_fp)
    echonet_video = echonet.utils.loadvideo(str(echonet_video_fp)) # (colours, frames, height, width)

    # Since right mask comes from flipped version of video, we need to flip it back to normal (i.e. left-to-right)
    right_segment_mask = np.flip(right_segment_mask, axis=-1)

    # Keep track of the mistaken left ventricle segmentation by the "right" segmentation mask
    mistaken_left_mask = left_segment_mask & right_segment_mask
    # Subtract those mistaken left ventricle parts from the right segmentation
    right_only_mask = right_segment_mask ^ (mistaken_left_mask)

    # Get heights of left and right ventricles, check if right ventricle appears unreasonably large (i.e. probably including atrium accidentally!)
    left_heights = get_heights(left_segment_mask)
    right_heights = get_heights(right_only_mask)
    frames_with_too_large_RV_mask = right_heights > 0.8 * left_heights

    # Put colour channels at end to make it easier to assign pixel colours
    echonet_video = echonet_video.transpose((1, 2, 3, 0)) # I.e. now (frames, height, width, colours)
    echonet_video[left_segment_mask] = RED
    echonet_video[right_only_mask] = BLUE
    echonet_video[mistaken_left_mask] = GREEN
        
    border_thickness = 2
    echonet_video[frames_with_too_large_RV_mask, 0:border_thickness, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, -border_thickness:, :] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, 0:border_thickness] = ORANGE
    echonet_video[frames_with_too_large_RV_mask, :, -border_thickness:] = ORANGE

    # Transpose video back to original shape now and save
    echonet_video = echonet_video.transpose((3, 0, 1, 2)) # now (colours, frames, height, width) as before
    output_fp = OUTPUT_DIR / video_stem.with_suffix(".avi")

    echonet.utils.savevideo(str(output_fp), echonet_video, 30)

  2%|▏         | 24/1276 [00:02<02:10,  9.61it/s]


KeyboardInterrupt: 

# Filtering out garbage data
Here, we'll be trying to discard any videos that are not A4C views.

In [8]:
def make_line(p1, p2, xs):
    x1, y1 = p1
    x2, y2 = p2
    
    gradient = (y2 - y1) / (x2 - x1)
    ys = gradient * (xs - x1) + y1
    ys = np.int0(ys)
    
    coords_list = list(zip(xs, ys))
    return coords_list

In [9]:
VIDEONAME = "0X1A58C9DFE12C7953"

LEFT_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/all-patients/segmentation_masks/{VIDEONAME}.npy")
FLIPPED_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/flipped/segmentation_masks/{VIDEONAME}.npy")
ECHONET_VIDEO_FP = Path(f"/home/lex/data/echonet-data/Videos/{VIDEONAME}.avi")

left_segmentation_mask = np.load(LEFT_SEGMENTATION_MASK_FP)
flipped_segmentation_mask = np.load(FLIPPED_SEGMENTATION_MASK_FP)
flipped_segmentation_mask = np.flip(flipped_segmentation_mask, -1)
intersection_mask = left_segmentation_mask & flipped_segmentation_mask
# Subtract those mistaken left ventricle parts from the right segmentation
right_segmentation_mask = flipped_segmentation_mask ^ intersection_mask
# right_segmentation_mask = right_segmentation_mask.astype(np.uint8) * 255

echonet_video = echonet.utils.loadvideo(str(ECHONET_VIDEO_FP))
echonet_video = echonet_video.transpose((1, 2, 3, 0))

# Calculate RV areas
right_segmentation_mask_image = right_segmentation_mask.astype(np.uint8) * 255
right_contours_list = [cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) for frame in right_segmentation_mask_image]
right_contours_list = [x[0] for x in right_contours_list]

right_max_areas = []
for i, right_contours in enumerate(right_contours_list):
    print(f"i = {i}")
    right_areas = [cv2.contourArea(contour) for contour in right_contours]
    max_area = max(right_areas)
    right_max_areas.append(max_area)

num_frames, height, width = flipped_segmentation_mask.shape

WINDOW = "Mask"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True

prev_area = None
while True:
    if i >= num_frames:
        i = 0
        print("Video looped!")
    elif i < 0:
        i = num_frames - 1

    frame = echonet_video[i].copy()
    frame_left_segmentation_mask = left_segmentation_mask[i]
    frame_right_segmentation_mask = right_segmentation_mask[i].copy()
    frame_intersection_segmentation_mask = intersection_mask[i]
    frame[frame_left_segmentation_mask] = RED
    frame[frame_intersection_segmentation_mask] = GREEN

    frame_left_segmentation_mask = frame_left_segmentation_mask.astype(np.uint8) * 255
    left_contours, hierarchy = cv2.findContours(frame_left_segmentation_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    left_contour = left_contours[0]

    min_area_rect = cv2.minAreaRect(left_contour)
    angle = min_area_rect[-1]
    box = np.int0(cv2.boxPoints(min_area_rect))
    cv2.drawContours(frame,[box],0,(0,0,255))

    top_left_LV, top_right_LV, bottom_right_LV, bottom_left_LV = box
    cutoff_x = np.int0(np.arange(0, frame.shape[1]))
    bottom_cutoff_points = make_line(bottom_left_LV, bottom_right_LV, cutoff_x)
    cv2.line(frame, bottom_cutoff_points[0], bottom_cutoff_points[-1], (0,0,255))
    
    right_cutoff_points_untrimmed = make_line(bottom_left_LV, top_left_LV, cutoff_x)
    right_cutoff_points = []
    for x, y in right_cutoff_points_untrimmed:
        if y >=0 and y < frame.shape[1]:
            right_cutoff_points.append((x, y))

    cv2.line(frame, right_cutoff_points[0], right_cutoff_points[-1], (0,0,255))

    for x, y in bottom_cutoff_points:
        frame_right_segmentation_mask[y:, x] = False

    for x, y in right_cutoff_points:
        frame_right_segmentation_mask[:y, x] = False

    frame[frame_right_segmentation_mask] = BLUE

    # frame_right_segmentation_mask = frame_right_segmentation_mask.astype(np.uint8) * 255
    # right_contours, hierarchy = cv2.findContours(frame_right_segmentation_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # right_areas = [cv2.contourArea(contour) for contour in right_contours]
    # max_area = max(right_areas)

    # if prev_area is not None:
    #     growth = max_area / prev_area
    #     print(f"RV Area growth: {growth:.2f}")
    this_area = right_max_areas[i]
    if i > 0: 
        prev_area = right_max_areas[i - 1]
        growth = this_area/ prev_area
        print(f"RV Area growth: {growth:.2f}")
        if growth > 1.2:
            frame[:10, :] = ORANGE

    top_border = np.zeros((height // 8, width, 3), dtype=frame.dtype)
    top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
    cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(255, 0, 0))
    frame = np.concatenate([top_border, frame], axis=0)
    frame = np.flip(frame, -1)
    cv2.imshow(WINDOW, frame)

    # prev_area = max_area

    # right_mask = cv2.cvtColor(right_mask, cv2.COLOR_GRAY2RGB)
    # cv2.drawConto|urs(right_mask, [max_contour], -1, (0,0,255), 1)

    # min_area_rect = cv2.minAreaRect(max_contour)
    # box = np.int0(cv2.boxPoints(min_area_rect))
    # cv2.drawContours(right_mask,[box],0,(0,0,255))

    
    # right_mask = np.concatenate([top_border, right_mask], axis=0)
    # cv2.imshow("Mask", right_mask)

    keypress = cv2.waitKey(50) & 0xFF
    if keypress == ord('q'):
        break
    elif keypress == ord(' '):
        is_playing = not is_playing
    elif keypress == ord('a'):
        i -= 1
    elif keypress == ord('d'):
        i += 1
    else:
        if is_playing:
            i += 1

cv2.destroyAllWindows()

i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42
i = 43
i = 44
i = 45
i = 46
i = 47
i = 48
i = 49
i = 50
i = 51
i = 52
i = 53
i = 54
i = 55
i = 56
i = 57
i = 58
i = 59
i = 60
i = 61
i = 62
i = 63
i = 64
i = 65
i = 66
i = 67
i = 68
i = 69
i = 70
i = 71
i = 72
i = 73
i = 74
i = 75
i = 76
i = 77
i = 78
i = 79
i = 80
i = 81
i = 82
i = 83
i = 84
i = 85
i = 86
i = 87
i = 88
i = 89
i = 90
i = 91
i = 92
i = 93
i = 94
i = 95
i = 96
i = 97
i = 98
i = 99
i = 100
i = 101
i = 102
i = 103
i = 104
i = 105
i = 106
i = 107
RV Area growth: 0.93


/tmp/ipykernel_435515/1551265270.py:59: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(cv2.boxPoints(min_area_rect))
/tmp/ipykernel_435515/1551265270.py:63: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  cutoff_x = np.int0(np.arange(0, frame.shape[1]))
/tmp/ipykernel_435515/1473003995.py:7: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  ys = np.int0(ys)


RV Area growth: 1.01
RV Area growth: 1.01
RV Area growth: 0.98
RV Area growth: 0.99
RV Area growth: 0.97
RV Area growth: 1.07
RV Area growth: 1.04
RV Area growth: 0.92
RV Area growth: 0.98
RV Area growth: 0.95
RV Area growth: 0.95
RV Area growth: 0.98
RV Area growth: 0.90
RV Area growth: 0.94
RV Area growth: 0.98
RV Area growth: 1.07
RV Area growth: 0.95
RV Area growth: 0.99
RV Area growth: 1.06
RV Area growth: 1.55
RV Area growth: 0.89
RV Area growth: 1.14
RV Area growth: 1.17
RV Area growth: 1.04
RV Area growth: 1.05
RV Area growth: 1.02
RV Area growth: 0.96
RV Area growth: 0.98
RV Area growth: 1.00
RV Area growth: 0.94
RV Area growth: 0.96
RV Area growth: 0.95
RV Area growth: 1.00
RV Area growth: 0.81
RV Area growth: 0.94
RV Area growth: 1.11
RV Area growth: 1.00
RV Area growth: 0.99
RV Area growth: 1.00
RV Area growth: 1.06
RV Area growth: 0.94
RV Area growth: 0.84
RV Area growth: 0.97
RV Area growth: 1.16
RV Area growth: 0.78
RV Area growth: 1.21
RV Area growth: 0.84
RV Area growt

In [ ]:
WINDOW = "Mask"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True

frame = np.zeros((118, 118), dtype=np.uint8)
frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

while True:
    if i >= num_frames:
        i = 0
        print("Video looped!")
    elif i < 0:
        i = num_frames - 1

    frame = cv2.rectangle(frame, corner1, corner2, RED)
    cv2.imshow(WINDOW, frame)

    keypress = cv2.waitKey(50) & 0xFF
    if keypress == ord('q'):
        break
    elif keypress == ord(' '):
        is_playing = not is_playing
    elif keypress == ord('a'):
        i -= 1
    elif keypress == ord('d'):
        i += 1
    else:
        if is_playing:
            i += 1

cv2.destroyAllWindows()

In [6]:
tuple(int(GREEN))[0].dtype

TypeError: only size-1 arrays can be converted to Python scalars